In [1]:
import csv
import numpy as np
from random import shuffle
import scipy
from sklearn.metrics import mean_squared_error

In [2]:
f1 = open('./train.csv','rb')
f2 = open('./dev.csv','rb')
f3 = open('./test.csv','rb')
reader = csv.reader(f1)
f1.readline()
train = []
dev = []
test = []
for row in reader:
    train.append((row[0],row[1],row[2]))
reader = csv.reader(f2)
f2.readline()
for row in reader:
    dev.append((row[0],row[1],row[2]))
reader = csv.reader(f3)
f3.readline()
for row in reader:
    test.append((row[0],row[1],row[2]))
f1.close()
f2.close()
f3.close()

##Baseline
$b_{u,i}=\mu+b_{u}+b_{i}$<br><br>
$b_{u}=\frac{1}{\left | I_{u} \right |+\beta_{u}}\sum_{i\in I_{u}}^{ }(r_{u,i}-\mu)$<br><br>
$b_{i}=\frac{1}{\left | U_{i} \right |+\beta_{i}}\sum_{u\in U_{i}}^{ }(r_{u,i}-b_{u}-\mu)$<br><br>
$b_{u,i}$ is the predicted rating for user u on restaurant i<br>
$\mu$ is the average rating over all ratings in the system<br>
$r_{u,i}$ is the existant rating for user u on restaurant i<br>
$\left | I_{u} \right |$ is number of ratings made by user u<br>
$\left | U_{i} \right |$ is number of ratings made on restaurant i<br>
$\beta_{u}, \beta_{i}$ are regularization terms<br>

In [3]:
class baseline:
    bui = {}
    u = 0
    
    def __init__(self,betau,betai):
        self.betau = betau
        self.betai = betai
        
    def fit(self,train):
        user = {}
        movie = {}
        bu = {}
        bi = {}
        bui = {}
        n = 0
        for unit in set([x[0] for x in train]):
            user[int(unit)] = n
            n += 1
        n = 0
        for unit in set([x[1] for x in train]):
            movie[int(unit)] = n
            n += 1
        u = np.mean([int(x[2]) for x in train])
        self.u = u
        matrix = np.zeros((len(user),len(movie)))
        for data in train:
            matrix[user[int(data[0])],movie[int(data[1])]] = int(data[2])
        for usr in user.keys():
            bu[usr] = 1./((len(np.where(matrix[user[usr],] != 0)[0]))+self.betau)*np.sum([x-u for x in matrix[user[usr],] if x != 0])
        for mov in movie.keys():
            bi[mov] = 1./((len(np.where(matrix[:,movie[mov]] != 0)[0]))+self.betai)*np.sum([x-u for x in matrix[:,movie[mov]] if x != 0])
        for usr in user.keys():
            for mov in movie.keys():
                if matrix[user[usr],movie[mov]] == 0:
                    bui[(usr,mov)] = u + bu[usr] + bi[mov]
                else:
                    bui[(usr,mov)] = matrix[user[usr],movie[mov]]
        self.bui = bui
        return 'Baseline Model training finished beta u = '+str(self.betau)+' beta i = '+str(self.betai)
    
    def predict(self,test):
        result = []
        for line in test:
            if(int(line[0]),int(line[1])) not in self.bui:
                result.append(round(self.u))
            else:
                result.append(round(self.bui[(int(line[0]),int(line[1]))]))
        return result

In [4]:
bl = baseline(25,25)
bl.fit(train)

'Baseline Model training finished beta u = 25 beta i = 25'

In [5]:
predicted_ratings1 = bl.predict(dev)
predicted_ratings2 = bl.predict(test)

In [6]:
fs = open('./dev.predict','w')
for rating in predicted_ratings1:
    fs.write(str(int(rating))+'\n')
fs.close()
    
fs2 = open('./test.predict','w')
for rating in predicted_ratings2:
    fs2.write(str(int(rating))+'\n')
fs2.close()

In [8]:
ft1 = open('./dev.solution')
ft2 = open('./test.solution')
truth1 = []
truth2 = []
for rating in ft1:
    truth1.append(int(rating[0]))
for rating in ft2:
    truth2.append(int(rating[0]))

In [10]:
rmse_dev = mean_squared_error(predicted_ratings1,truth1)
rmse_test = mean_squared_error(predicted_ratings2,truth2)

In [12]:
print 'RMSE of prediction baseline on dataset dev is '+str(rmse_dev)
print 'RMSE of prediction baseline on dataset test is '+str(rmse_test)

RMSE of prediction baseline on dataset dev is 0.993664281455
RMSE of prediction baseline on dataset test is 1.00402504472
